# Debugging and Understanding DNNs using <font color=#ff6600>TensorBoard</font> (in Google Colab) 

## Following: [Quick guide to run Tensorboard in Google Colab](https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/)

## By Chengwei 

![Tensorboard](https://cdn-images-1.medium.com/max/1600/1*Ta8DXEzDBjtzLw7VeYwMyQ.png)
<br>



# * [MSTC](http://mstc.ssr.upm.es/big-data-track) and MUIT: <font size=5 color='green'>Deep Learning with Tensorflow & Keras</font>




---
# <font color=#ff6600> TensorBoard</font>: 

## [Visualizing Learning](https://www.tensorflow.org/guide/summaries_and_tensorboard)

- ## A Visualization Tool to understand, debug, and optimize TensorFlow programs.

- ## You can use TensorBoard to visualize your TensorFlow graph, plot quantitative metrics about the execution of your graph, and show additional data like images that pass through it. When TensorBoard is fully configured, it looks like this:

## in <font color=FF103> Keras</font> : you have you use TensorBoard [callbacks](https://keras.io/callbacks/)

    * A callback is a set of functions to be applied at given stages of the training procedure.
    
    * You can use callbacks to get a view on internal states and statistics of the model during training.

<font size=4 color=FF103>TensorBoard callback</font>

<font size=4>This callback writes a log for TensorBoard, which allows you to visualize dynamic graphs of your training and test metrics, as well as activation histograms for the different layers in your model.

      tensorboard --logdir=/full_path_to_your_logs



---

## How to access the TensorBoard page from our local machine?

- ## We are going to use a free service named <font color=BB9903>ngrok</font> to tunnel the connection to your local machine.

<font size=5>Here is a graph to show how it works.</font>

![ngrog](https://gitcdn.xyz/cdn/Tony607/blog_statics/d425c3fe4cf0d92067572e25ae6cc3198d51936b//images/ngrok/ngrok.jpg)



In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

## Run <font color=#ff6600>TensorBoard</font>

- ### *logdir* should be provided with the same logs directory path that you provided in your training code (You can name it whatever you want)

- ### Use host and port (6006) for accssing the Tensorboard dashboard.

- ### The dashboard won’t display the plots until the model training begins.

In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

## Run <font color=BB9903>ngrok</font>

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

## Get URL
Run the next cell to start the training before open the url.

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## Remember that 
- ### The dashboard won’t display the plots until the model training begins.


---

## Run a Keras model with <font color=#ff6600>Tensorboard</font>

### [Train a simple deep NN on the MNIST dataset](https://github.com/keras-team/keras/blob/master/examples/mnist_mlp.py)


In [0]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.callbacks import TensorBoard

batch_size = 128
num_classes = 10
epochs = 20

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')


print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))

model.add(Dense(num_classes, activation='softmax'))

model.summary()


- ### you only can see the graph but it is no easy: https://www.tensorflow.org/guide/summaries_and_tensorboard 

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])


tbCallBack = TensorBoard(log_dir='./log', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tbCallBack])




score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Optionally check tensorboard and <font color=BB9903>ngrok</font> PIDs and stop them

In [0]:
!ps aux | grep tensorboard

In [0]:
!ps aux | grep ngrok

In [0]:
# !kill 881

## Optionally clear the <font color=#ff6600>Tensorboard</font> records directory

In [0]:
ls -al ./log

In [0]:
#! rm ./log/events.out.tfevents.1542892772.ac243cb9284d

In [0]:
#!rm -rf ./log

In [0]:
#!rm -rf ngrok-stable-linux-amd64.zip*